# Q1: Validate/Coerce Untrusted JSON from LLM

You receive untrusted JSON from an LLM. Validate/coerce it to the schema below.

## Target Schema

```json
{
  "action": "search" | "answer",
  "q": non-empty str,  // required iff action == "search"
  "k": int in [1, 5]  // optional (default: 3)
}
```

## Implementation Requirements

Implement the following method:

```python
from typing import Any, Dict, List, Tuple

def validate_tool_call(payload: Dict[str, Any]) -> Tuple[Dict[str, Any], List[str]]:
    """
    Returns (clean, errors). 'clean' strictly follows the schema with defaults applied.

    Rules:
    - Trim strings; coerce numeric strings to ints.
    - Remove unknown keys.
    - If action=='answer', ignore 'q' if present (no error).
    - On fatal errors (e.g., missing/invalid 'action', or missing/empty 'q' for search),
      return ({}, errors).
    """
```


In [ ]:
from typing import Any, Dict, List, Literal, Optional, Tuple, TypedDict, cast
from typing_extensions import NotRequired

ActionLiteral = Literal["search", "answer"]
ALLOWED_ACTIONS: set[str] = {"search", "answer"}
DEFAULT_K = 3
MIN_K = 1
MAX_K = 5


class LLMResponse(TypedDict):
    """Input payload schema - matches target schema structure."""
    action: ActionLiteral  # required
    q: NotRequired[str]  # required if action == "search"
    k: NotRequired[int]  # optional, must be in [1, 5]


class ValidatedSearchCall(TypedDict):
    action: Literal["search"]
    q: str
    k: int


class ValidatedAnswerCall(TypedDict):
    action: Literal["answer"]
    k: int


ValidatedToolCall = ValidatedSearchCall | ValidatedAnswerCall


def _coerce_int(value: Any) -> Optional[int]:
    """Try to coerce supported values into an int; return None if impossible."""
    if isinstance(value, bool):
        return None
    if isinstance(value, int):
        return value
    if isinstance(value, str):
        stripped = value.strip()
        if not stripped:
            return None
        try:
            return int(stripped)
        except ValueError:
            return None
    return None


def _validate_action(value: Any, errors: List[str]) -> Optional[ActionLiteral]:
    if not isinstance(value, str):
        errors.append("action must be one of 'search' or 'answer'.")
        return None
    normalized = value.strip().lower()
    if normalized not in ALLOWED_ACTIONS:
        errors.append("action must be one of 'search' or 'answer'.")
        return None
    return cast(ActionLiteral, normalized)


def _validate_query(value: Any, errors: List[str]) -> Optional[str]:
    if value is None:
        errors.append("q is required when action is 'search'.")
        return None
    if not isinstance(value, str):
        errors.append("q must be a string when action is 'search'.")
        return None
    trimmed = value.strip()
    if not trimmed:
        errors.append("q must be a non-empty string when action is 'search'.")
        return None
    return trimmed


def _validate_k(value: Any, errors: List[str]) -> int:
    if value is None:
        return DEFAULT_K
    coerced = _coerce_int(value)
    if coerced is None:
        errors.append("k must be an integer between 1 and 5; defaulting to 3.")
        return DEFAULT_K
    if not (MIN_K <= coerced <= MAX_K):
        errors.append("k must be between 1 and 5; defaulting to 3.")
        return DEFAULT_K
    return coerced


def validate_tool_call(payload: Dict[str, Any]) -> Tuple[ValidatedToolCall | Dict[str, Any], List[str]]:
    """
    Returns (clean, errors). 'clean' strictly follows the schema with defaults applied.
    On fatal errors, returns ({}, errors).
    """
    errors: List[str] = []
    clean: Dict[str, Any] = {}

    action_value = _validate_action(payload.get("action"), errors)
    if action_value is None:
        return {}, errors
    clean["action"] = action_value

    if action_value == "search":
        query = _validate_query(payload.get("q"), errors)
        if query is None:
            return {}, errors
        clean["q"] = query

    clean["k"] = _validate_k(payload.get("k"), errors)

    return cast(ValidatedToolCall, clean), errors

